In [ ]:
import os
from pymongo import MongoClient

### Connection to MongoDB

In [ ]:
mongodb_uri = os.environ.get('MONGODB_URI')
database_name = os.environ.get('DATABASE_NAME')

client = MongoClient(mongodb_uri)
database = client[database_name]

#### Connecting to database collections

In [ ]:
videos = database['videos']     
audio_descriptions = database['audio_descriptions']

##### Updating video field in audio descriptions collection based on audio descriptions field of video collection

In [ ]:
video_pipeline = [
    {
        '$match': {
            'audio_descriptions': {
                '$exists': True, 
                '$ne': []
            }
        }
    }, {
        '$unwind': {
            'path': '$audio_descriptions'
        }
    }, {
        '$project': {
            '_id': 0, 
            'audio_description_id': '$audio_descriptions', 
            'video_id': '$_id'
        }
    }, {
        '$out': 'AD_Video'      # new collection
    }
]

videos.aggregate(video_pipeline)

In [ ]:
ad_pipeline = [
    {
        '$lookup': {
            'from': 'AD_Video', 
            'localField': '_id', 
            'foreignField': 'audio_description_id', 
            'as': 'AD_Video'
        }
    }, {
        '$unwind': {
            'path': '$AD_Video'
        }
    }, {
        '$addFields': {
            'video': '$AD_Video.video_id'
        }
    }, {
        '$project': {
            'AD_Video': 0
        }
    }, {
        '$out': 'modified_audio_descriptions'  # new audio descriptions collection
    }
]

audio_descriptions.aggregate(ad_pipeline)